In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean , median
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost"])
baseline_data = np.array(baseLine_data)
stats = median(baseline_data[:,0])
catboost = median(baseline_data[:,1])

# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.003925,0.046379,0.278253,0.927583
1,0.015190,0.090338,0.550521,1.806768
2,0.003680,0.050162,0.270881,1.003233
3,0.002585,0.039138,0.226374,0.782752
4,0.010736,0.082382,0.462688,1.647635
5,0.004975,0.057724,0.315023,1.154474
6,0.006896,0.066599,0.370883,1.331986
7,0.004594,0.050541,0.302654,1.010813
8,0.007402,0.065142,0.384195,1.302832
9,0.007516,0.063800,0.385625,1.275995


MEDIAN:
MSE                   0.005936
MAE                   0.060762
Euclidean Distance    0.342953
Manhattan Distance    1.215234
dtype: float64
0.06076171525809913


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [5]:
book = sb.read_notebooks("./ABC_GAN_Catboost")
paramVal = [[1,1],[1,0.1],[1,0.01],[0.1,1],[0.1,0.1],[0.1,0.01],[0.01,1],[0.01,0.1],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [6]:

data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.07855,1.36006,0.11787,0.03983,0.04973
1,1,1,0.15301,1.08836,0.10816,0.05588,0.06058
2,1,1,0.14898,1.12789,0.90731,0.07048,0.05416
3,1,1,0.19753,1.18692,0.11900,0.07974,0.04311
4,1,1,0.11723,0.85215,0.88979,0.09667,0.08456
5,1,1,0.15040,1.20445,0.09598,0.06731,0.05081
6,1,1,0.15466,0.95156,0.31636,0.09388,0.05459


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.150399
ABC pre-generator MAE            1.127885
Skip Node weight                 0.118999
ABC GAN MAE                      0.070475
ABC_GAN MAE (skip connection)    0.054163
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.1,0.08664,1.08078,0.22811,0.06467,0.05341
1,1,0.1,0.12254,0.96230,0.07545,0.09036,0.08981
2,1,0.1,0.09482,0.99885,0.08496,0.08375,0.05871
3,1,0.1,0.20895,0.96605,0.62171,0.08128,0.11557
4,1,0.1,0.25964,1.14851,0.15538,0.11147,0.12880
5,1,0.1,0.10261,1.02987,0.06901,0.05347,0.05544
6,1,0.1,0.24933,0.96000,0.07279,0.14778,0.15382


Variance                         1.000000
Bias                             0.100000
Prior Model MAE                  0.122537
ABC pre-generator MAE            0.998852
Skip Node weight                 0.084956
ABC GAN MAE                      0.083755
ABC_GAN MAE (skip connection)    0.089808
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.01,0.10032,1.04452,0.07525,0.07946,0.09370
1,1,0.01,0.15789,0.95218,0.86445,0.11203,0.11231
2,1,0.01,0.06501,0.99964,0.19704,0.06338,0.04880
3,1,0.01,0.10682,0.98862,0.75384,0.05058,0.04444
4,1,0.01,0.14664,1.01787,0.07657,0.04879,0.13630
5,1,0.01,0.14552,0.97122,0.30354,0.08534,0.12470
6,1,0.01,0.10720,0.99729,0.06635,0.07713,0.09104


...

Variance                         0.010000
Bias                             1.000000
Prior Model MAE                  0.144034
ABC pre-generator MAE            0.829097
Skip Node weight                 0.648910
ABC GAN MAE                      0.063192
ABC_GAN MAE (skip connection)    0.059837
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.1,0.08611,0.10014,0.11448,0.03919,0.04223
1,0.01,0.1,0.14182,0.16452,0.38924,0.07521,0.08046
2,0.01,0.1,0.08366,0.13291,0.85842,0.06312,0.03845
3,0.01,0.1,0.15718,0.19995,0.58195,0.09116,0.09086
4,0.01,0.1,0.08675,0.14506,0.03570,0.04680,0.04430
5,0.01,0.1,0.14809,0.15994,0.15341,0.08700,0.08208
6,0.01,0.1,0.17979,0.24141,0.09049,0.09387,0.06309


Variance                         0.010000
Bias                             0.100000
Prior Model MAE                  0.141818
ABC pre-generator MAE            0.159936
Skip Node weight                 0.153409
ABC GAN MAE                      0.075211
ABC_GAN MAE (skip connection)    0.063094
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.19134,0.18994,0.00039,0.11355,0.19216
1,0.01,0.01,0.23222,0.23572,0.01917,0.15256,0.22942
2,0.01,0.01,0.15958,0.16099,0.65389,0.12704,0.14181
3,0.01,0.01,0.07425,0.07819,0.01657,0.05087,0.06998
4,0.01,0.01,0.19488,0.19595,0.00000,0.10992,0.19472
5,0.01,0.01,0.13120,0.13433,0.01152,0.10145,0.13048
6,0.01,0.01,0.11860,0.12004,0.06184,0.08086,0.12044


Variance                         0.010000
Bias                             0.010000
Prior Model MAE                  0.159576
ABC pre-generator MAE            0.160987
Skip Node weight                 0.016569
ABC GAN MAE                      0.109923
ABC_GAN MAE (skip connection)    0.141806
dtype: float64
-------------------------------------------------------------


In [7]:
# Display Catboost Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([paramVal[i][0], paramVal[i][1],catboost,median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['Variance','Bias','Catboost','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

,Variance,Bias,Catboost,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,1.00,1.00,0.03605,1.12789,0.07048,0.05416,0.11900
1,1.00,0.10,0.03605,0.99885,0.08375,0.08981,0.08496
2,1.00,0.01,0.03605,0.99729,0.07713,0.09370,0.19704
3,0.10,1.00,0.03605,0.79639,0.07749,0.06772,0.18788
4,0.10,0.10,0.03605,0.21265,0.07579,0.08006,0.08516
5,0.10,0.01,0.03605,0.15207,0.09983,0.12781,0.19254
6,0.01,1.00,0.03605,0.82910,0.06319,0.05984,0.64891
7,0.01,0.10,0.03605,0.15994,0.07521,0.06309,0.15341
8,0.01,0.01,0.03605,0.16099,0.10992,0.14181,0.01657


## ABC Pre-generator - Stats 


In [8]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [[1,1],[0.1,1],[0.01,1],[1,0.1],[0.1,0.1],[0.01,0.1],[1,0.01],[0.1,0.01],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [9]:
data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.23813,1.28106,0.80094,0.05578,0.03611
1,1,1,0.22102,1.14056,0.99307,0.08598,0.04595
2,1,1,0.27459,1.08816,0.98481,0.07290,0.05594
3,1,1,0.28813,1.31808,0.83078,0.08666,0.05056
4,1,1,0.34480,1.12133,0.91197,0.10008,0.06026
5,1,1,0.24110,1.03853,0.89858,0.06163,0.06707
6,1,1,0.33800,1.08370,0.85938,0.10560,0.07152
7,1,1,0.33133,1.30251,0.87432,0.09254,0.05680
8,1,1,0.27649,1.56975,0.86954,0.08778,0.05912
9,1,1,0.24126,1.44175,0.84671,0.10785,0.06424


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.275544
ABC pre-generator MAE            1.210808
Skip Node weight                 0.871929
ABC GAN MAE                      0.087218
ABC_GAN MAE (skip connection)    0.057961
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,1,0.25161,1.00356,0.80055,0.10176,0.07265
1,0.1,1,0.23106,0.97778,0.94645,0.08995,0.06388
2,0.1,1,0.19174,0.85203,0.90555,0.06332,0.05827
3,0.1,1,0.32052,0.85877,0.84012,0.09127,0.05755
4,0.1,1,0.35152,1.15790,0.96313,0.12788,0.07187
5,0.1,1,0.26972,0.88087,0.89831,0.15212,0.04405
6,0.1,1,0.22885,1.01589,0.92057,0.13595,0.06548
7,0.1,1,0.24305,0.72427,0.97428,0.05745,0.04699
8,0.1,1,0.29900,0.72984,0.88633,0.07252,0.06855
9,0.1,1,0.25311,0.99157,0.83004,0.07409,0.06332


Variance                         0.100000
Bias                             1.000000
Prior Model MAE                  0.252361
ABC pre-generator MAE            0.929323
Skip Node weight                 0.901929
ABC GAN MAE                      0.090611
ABC_GAN MAE (skip connection)    0.063601
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,1,0.32042,1.04047,0.81824,0.15056,0.05376
1,0.01,1,0.28806,0.95742,0.83305,0.09888,0.06393
2,0.01,1,0.29300,0.82354,0.83751,0.09255,0.05515
3,0.01,1,0.26589,0.70296,0.99301,0.08804,0.07910
4,0.01,1,0.34727,0.83058,0.94527,0.11802,0.11185
5,0.01,1,0.28665,0.94167,0.85447,0.13636,0.12575
6,0.01,1,0.19762,0.93788,0.85314,0.08796,0.05882
7,0.01,1,0.33222,0.96593,0.90723,0.17413,0.06964
8,0.01,1,0.31160,0.93358,0.90637,0.04920,0.04424
9,0.01,1,0.26259,1.09724,0.90015,0.11388,0.05591


...

Variance                         1.000000
Bias                             0.010000
Prior Model MAE                  0.314033
ABC pre-generator MAE            0.997419
Skip Node weight                 0.310780
ABC GAN MAE                      0.100257
ABC_GAN MAE (skip connection)    0.060356
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.01,0.29040,0.30087,0.18846,0.18791,105.53128
1,0.1,0.01,0.20689,0.22834,0.12998,0.09103,0.05241
2,0.1,0.01,0.22965,0.27914,0.20615,0.09268,0.07924
3,0.1,0.01,0.26198,0.30295,0.30962,0.12064,0.11256
4,0.1,0.01,0.24439,0.26979,0.17537,0.07644,0.05179
5,0.1,0.01,0.30451,0.33510,0.21017,0.11085,0.05231
6,0.1,0.01,0.17371,0.17706,0.13925,0.08194,0.05806
7,0.1,0.01,0.27235,0.29969,0.14823,0.08227,0.07366
8,0.1,0.01,0.25453,0.26400,0.16646,0.04820,0.03724
9,0.1,0.01,0.25111,0.25722,0.10377,0.13528,43.47140


Variance                         0.100000
Bias                             0.010000
Prior Model MAE                  0.252820
ABC pre-generator MAE            0.274466
Skip Node weight                 0.170914
ABC GAN MAE                      0.091859
ABC_GAN MAE (skip connection)    0.065858
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.27701,0.27601,0.15135,0.21197,0.09347
1,0.01,0.01,0.33105,0.33298,0.12219,0.08594,0.09629
2,0.01,0.01,0.25052,0.25091,0.20049,0.14058,0.05829
3,0.01,0.01,0.24286,0.24847,0.16389,0.12005,0.03961
4,0.01,0.01,0.26252,0.26526,0.29397,0.10805,0.10803
5,0.01,0.01,0.31213,0.31339,0.24219,0.08427,0.08858
6,0.01,0.01,0.20266,0.20573,0.17417,0.10652,0.05460
7,0.01,0.01,0.26714,0.26767,0.11562,0.09334,0.04925
8,0.01,0.01,0.38525,0.37973,0.15593,0.10218,0.10349
9,0.01,0.01,0.38200,0.38359,0.20646,0.12734,0.10027


Variance                         0.010000
Bias                             0.010000
Prior Model MAE                  0.272074
ABC pre-generator MAE            0.271841
Skip Node weight                 0.169029
ABC GAN MAE                      0.107284
ABC_GAN MAE (skip connection)    0.091025
dtype: float64
-------------------------------------------------------------


In [10]:
# Display Stats Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([paramVal[i][0], paramVal[i][1],stats,median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['Variance','Bias','Stats','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

,Variance,Bias,Stats,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,1.00,1.00,0.26665,1.21081,0.08722,0.05796,0.87193
1,0.10,1.00,0.26665,0.92932,0.09061,0.06360,0.90193
2,0.01,1.00,0.26665,0.93977,0.10638,0.06138,0.87731
3,1.00,0.10,0.26665,0.95306,0.09608,0.05284,0.39998
4,0.10,0.10,0.26665,0.35147,0.11277,0.05688,0.24184
5,0.01,0.10,0.26665,0.26200,0.07843,0.04462,0.21175
6,1.00,0.01,0.26665,0.99742,0.10026,0.06036,0.31078
7,0.10,0.01,0.26665,0.27447,0.09186,0.06586,0.17091
8,0.01,0.01,0.26665,0.27184,0.10728,0.09103,0.16903
